In [91]:
import scipy
import cartoframes
import pandas as pd
import pysal
import geopandas as gpd
import networkx as nx
import numpy as np

In [3]:
cc = cartoframes.CartoContext()

In [37]:
cc.creds

Credentials(username=michellemho-carto, key=29a2786e7462e8cdab49674be25159fce8939048, base_url=https://michellemho-carto.carto.com)

In [39]:
collision_join_geom = cc.read('collision_join_geom', decode_geom=True)

In [40]:
collision_join_geom.columns

Index(['bike_trafdir', 'bikelane', 'collision_count', 'featuretyp',
       'nodeidfrom', 'nodeidto', 'nonped', 'number_park_lanes',
       'number_total_lanes', 'number_travel_lanes', 'posted_speed',
       'shape_length', 'snow_priority', 'streetcode', 'streetwidth_max',
       'sum_cyclist_injured', 'sum_cyclist_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'trafdir', 'geometry'],
      dtype='object')

In [41]:
collision_join_geo = gpd.GeoDataFrame(collision_join_geom)

In [121]:
grouped_collision = pd.DataFrame(collision_join_geo.groupby(['nodeidfrom', 'nodeidto'])['collision_count'].sum()).reset_index()

In [188]:
collision_join_geo.head()

,bike_trafdir,bikelane,collision_count,featuretyp,nodeidfrom,nodeidto,nonped,number_park_lanes,number_total_lanes,number_travel_lanes,...,sum_fatalities,sum_injuries,sum_motorists_injured,sum_motorists_killed,sum_pedestrian_injured,sum_pedestrian_killed,the_geom,the_geom_str,trafdir,geometry
cartodb_id,,,,,,,,,,,,,,,,,,,,,
73790,TF,2,0,0,9007738,0015363,ped,1,2,1,...,0,0,0,0,0,0,0105000020E6100000010000000102000000020000007A...,0105000000010000000102000000020000007a18541d60...,A,(LINESTRING (-74.00586636745894 40.71065027182...
93059,TF,1,0,0,0082632,0083439,ped,1,2,1,...,0,0,0,0,0,0,0105000020E610000001000000010200000002000000AE...,010500000001000000010200000002000000ae10707330...,A,(LINESTRING (-73.90920721000541 40.74226152513...
135132,FT,3,0,0,0007993,0007994,ped,2,3,1,...,0,0,0,0,0,0,0105000020E610000001000000010200000002000000C3...,010500000001000000010200000002000000c3fa964eac...,W,(LINESTRING (-74.10426678413482 40.57897144021...
642,No bike_trafdir,No Bike Lane,0,0,0084655,0045050,ped,0,2,2,...,0,0,0,0,0,0,0105000020E61000000100000001020000000200000002...,01050000000100000001020000000200000002c5c75a0d...,T,(LINESTRING (-73.90706510075327 40.89927414148...
643,No bike_trafdir,No Bike Lane,0,0,0084773,0049421,ped,2,4,2,...,0,0,0,0,0,0,0105000020E610000001000000010200000002000000EF...,010500000001000000010200000002000000efcbc16cb9...,T,(LINESTRING (-73.90194243359996 40.90117827302...


In [178]:
adjacency_dict = {}
for row in collision_join_geo.iterrows():
    neighbors = []
    for other_rows in collision_join_geo.iterrows():
        if (row[1]['nodeidfrom'] == other_rows[1]['nodeidfrom'] or row[1]['nodeidfrom'] == other_rows[1]['nodeidto'] or row[1]['nodeidto'] == other_rows[1]['nodeidfrom'] or row[1]['nodeidto'] == other_rows[1]['nodeidto']) and other_rows[0] != row[0]:
            neighbors.append(other_rows[0])
    adjacency_dict[row[0]] = neighbors

KeyboardInterrupt: 

In [239]:
dictionary= {}
for node_id, dest_ids in collision_join_geo[['nodeidfrom', 'nodeidto']].groupby('nodeidfrom'):
    dictionary[node_id]= list(dest_ids.nodeidto.values)
dictionary

{'0000014': ['0000016'],
 '0000017': ['0071666'],
 '0000018': ['0070260', '9039351'],
 '0000019': ['0000014', '0071662'],
 '0000022': ['0000014'],
 '0000023': ['0000022'],
 '0000025': ['0000023'],
 '0000026': ['0000019', '0000022', '0000022'],
 '0000027': ['0000025', '0000026'],
 '0000031': ['0000075'],
 '0000048': ['0071669', '0071669'],
 '0000051': ['0000053'],
 '0000052': ['0000053', '0000051'],
 '0000054': ['0000058', '0000052'],
 '0000055': ['0000054', '0000112'],
 '0000056': ['0000018', '0000018', '0000052'],
 '0000057': ['0000056', '0000056'],
 '0000059': ['0000057', '0000057'],
 '0000060': ['0000055'],
 '0000061': ['0000116'],
 '0000062': ['0000059', '0000059'],
 '0000063': ['0000056'],
 '0000064': ['0000063', '0000027'],
 '0000065': ['0000057', '0000063'],
 '0000066': ['0000059', '0000065'],
 '0000067': ['0000066', '0065574'],
 '0000068': ['0065575'],
 '0000069': ['0000067', '0000070'],
 '0000070': ['0000062', '0000066'],
 '0000071': ['0000072', '0000069'],
 '0000072': ['00000

In [240]:
len(dictionary)

91813

In [243]:
collision_join_geo.nodeidfrom.nunique()

91813

In [218]:
adjacency_dict = {}
for row in collision_join_geo.iterrows():
    row_id = row[0]
    nodefrom = row[1]['nodeidfrom']
    nodeto = row[1]['nodeidto']
    neighbors = collision_join_geo[(collision_join_geo['nodeidfrom'] == nodefrom) | (collision_join_geo['nodeidto'] == nodeto) | (collision_join_geo['nodeidfrom'] == nodeto) | (collision_join_geo['nodeidto'] == nodefrom)]
    neighbors_id = list(neighbors.index.values)
    adjacency_dict[row_id] = neighbors_id

KeyboardInterrupt: 

In [220]:
len(adjacency_dict)

25611

In [123]:
G = nx.from_pandas_edgelist(grouped_collision, 'nodeidfrom', 'nodeidto', edge_attr=['collision_count'])

In [124]:
G.number_of_edges()

130472

In [125]:
G.number_of_nodes()

94627

In [126]:
adj = nx.adjacency_matrix(G)

In [127]:
adj.shape

(94627, 94627)

In [128]:
adj.todense()

matrix([[0, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [129]:
sparse_lobster = pysal.weights.WSP(nx.adj_matrix(G))
dense_lobster = sparse_lobster.to_W()

In [130]:
dense_lobster.histogram

[(1, 4062), (2, 41734), (3, 22608), (4, 25614), (5, 527), (6, 75), (7, 7)]

In [131]:
dense_lobster.n

94627

# Local Moran's I

In [154]:
fromnodes = pd.DataFrame(collision_join_geo.groupby(['nodeidfrom'])['collision_count'].sum()).reset_index()
fromnodes.rename(columns={'nodeidfrom':'nodeid'},inplace=True)

In [155]:
tonodes = pd.DataFrame(collision_join_geo.groupby(['nodeidto'])['collision_count'].sum()).reset_index()
tonodes.rename(columns={'nodeidto':'nodeid'},inplace=True)

In [161]:
nodes = pd.merge(tonodes, fromnodes, on=['nodeid']).set_index(['nodeid']).sum(axis=1)

In [162]:
nodes.shape

(89723,)

In [93]:
f = pysal.open(pysal.examples.get_path("sids2.dbf"))
b = np.array(f.by_col('BIR79'))
e = np.array(f.by_col('SID79'))
w = pysal.open(pysal.examples.get_path("sids2.gal")).read()
np.random.seed(12345)
lm = pysal.esda.moran.Moran_Local_Rate(e, b, w)
lm.Is[:10]

array([-0.13452366, -1.21133985,  0.05019761,  0.06127125, -0.12627466,
        0.23497679,  0.26345855, -0.00951288, -0.01517879, -0.34513514])

In [94]:
b.shape

(100,)

In [96]:
w.n

100

In [88]:
b = list(shape_length.values())

In [89]:
lm = pysal.esda.moran.Moran_Local_Rate(e, b, dense_lobster)

ValueError: dimension mismatch

In [230]:
import osmnx as ox
%matplotlib inline

nyc = ox.graph_from_place('New York City, New York, USA', network_type='drive')
# ox.plot_graph(ox.project_graph(nyc))

In [225]:
nyc_adj = nx.adj_matrix(nyc)

In [231]:
ox.save_graph_shapefile(nyc, filename='nyc-network-shape')

In [38]:
# Number of segments without collisions
y[y == 0].shape

(73177,)

In [39]:
# Number of segments with collisions
y[y > 0].shape

(46129,)

In [41]:
data.columns

Index(['bike_lane', 'bike_trafd', 'borocode', 'collision_count', 'date_creat',
       'date_modif', 'density_collision', 'density_cyclist_injured',
       'density_cyclist_killed', 'density_motorist_injured',
       'density_motorist_killed', 'density_pedestrian_injured',
       'density_pedestrian_killed', 'density_persons_injured',
       'density_persons_killed', 'frm_lvl_co', 'full_stree', 'index',
       'l_blkfc_id', 'l_high_hn', 'l_low_hn', 'l_zip', 'physicalid',
       'post_direc', 'post_modif', 'post_type', 'pre_direct', 'pre_modifi',
       'pre_type', 'r_blkfc_id', 'r_high_hn', 'r_low_hn', 'r_zip', 'rw_type',
       'shape_leng', 'snow_pri', 'st_label', 'st_name', 'st_width', 'status',
       'sum_cyclists_injured', 'sum_cyclists_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'time_creat', 'time_modif', 'to_lvl_co', 'trafdir'],
   

In [71]:
x = data[['f''bike_lane', 'bike_trafd', 'frm_lvl_co',
        'rw_type']]

In [74]:
x.head()

,bike_lane,bike_trafd,frm_lvl_co,rw_type
cartodb_id,,,,
12324,,,13,1
13189,,,13,1
79814,3,TF,13,1
8008,,,13,10
18899,,,13,1


In [75]:
x.bike_lane.unique()

array(['', '3', '1', '4', '2', '6', '8', '7', '9', '5'], dtype=object)